In [ ]:
import json
import os
import matplotlib.pyplot as plt
import pandas as pd
import folium
import ipinfo
import numpy as np

In [ ]:
def parse(filename):
    output = []
    with open(filename, "r") as file:
        for line in file.readlines():
            try:
                output.append(json.loads(line))
            except:
                pass
    return output

In [ ]:
def get_quic_usages(folders):
    output = []
    for folder in folders:
        for subfolder in [f for f in os.listdir(folder) if os.path.isdir(os.path.join(folder, f)) and f.startswith("rpm_")]:
            logfile = os.path.join(folder, subfolder, "proxy.log")
            if os.path.exists(logfile):
                try:
                    proxy_logs = parse(logfile)
                    if len(proxy_logs) > 0:
                        n_req = h3 = 0
                        for access in proxy_logs:
                            if access["request"]["method"] == "GET" or access["request"]["method"] == "POST":
                                if access["request"]["proto"] == "HTTP/3.0":
                                    h3 += 1
                                n_req += 1
                        output.append(100*h3/n_req)
                except:
                    pass
    return output


In [ ]:
def get_browsers(folders):
    df = pd.read_csv(os.path.join("data", "browsers.csv"))
    return df


browsers = get_browsers(["memoire-vps-output", "memoire-vps-output2"])
quic_usages = browsers["quic_usage"].values

In [ ]:
# quic_usages = get_quic_usages(["memoire-vps-output", "memoire-vps-output2"])

In [ ]:
plt.boxplot(quic_usages)
plt.ylabel("Percentage (%)")
plt.title("Percentage of connection that are using QUIC")

In [ ]:
sorted_values = np.sort(quic_usages)
cdf = np.arange(len(sorted_values)) / len(sorted_values)

plt.step(sorted_values, cdf)
plt.title("CDF of the percentage usage of QUIC connections")
plt.xlabel("Percentage of QUIC connections")
plt.ylabel("Cumulative Probability")

# plt.xlim(xmin=0)
plt.grid()
plt.savefig("output/quic_usage_cdf.pdf")

In [ ]:
plt.hist(sorted_values, bins=100)
# plt.title("Distribution of QUIC usage")
plt.xlabel("QUIC usage (%)")
plt.ylabel("Number of tests")
plt.savefig("output/quic_usage_pdf.pdf")

In [ ]:
sum([1 for p in quic_usages if p < 50])